In [51]:
# import os
from os import listdir
from os.path import isfile, join
import sys
import numpy as np
import cv2
import tensorflow as tf
import pickle
import facenet
#import align.detect_face
#import tfinit
import imageio
from scipy import misc
#from facenet import load_img
import matplotlib.pyplot as plt

lfw_batch_size        = 100
image_size            = 160
detect_multiple_faces = True
gpu_memory_fraction   = 0.6
margin                = 44

# Get user supplied values Loading the stored face embedding vectors for image retrieval
imagePath  = 'dataset/test/nguyen-manh.jpg'

img_path="/dataset/test"
#img_files = [f for f in listdir(img_path) if isfile(join(img_path, f))]

#img  = imageio.imread(imagePath)

with open('datasets/uit','rb') as f:
        feature_array = pickle.load(f)

model_exp = '../models/20180408-102900'
sess_tf, pnet, rnet, onet = tfinit.tf_init(model_exp, 'model-20180408-102900.meta', 'model-20180408-102900.ckpt-90', gpu_memory_fraction)
print('Loaded pre-trained models in', model_exp)

def align_face (img, pnet, rnet, onet):
        minsize = 20 # minimum size of face
        threshold = [ 0.7, 0.8, 0.8 ]  # three steps's threshold
        factor = 0.709 # scale factor

        if img.size == 0:
                print("empty array")
                return False,img,[0,0,0,0]

        if img.ndim<2:
            print('Unable to align')

        if img.ndim == 2:
            img = to_rgb(img)

        img = img[:,:,0:3]

        bounding_boxes, _ = align.detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)

        nrof_faces = bounding_boxes.shape[0]


        if nrof_faces==0:
            return False,img,[0,0,0,0]
        else:
            det = bounding_boxes[:,0:4]
            det_arr = []
            img_size = np.asarray(img.shape)[0:2]
            if nrof_faces>1:
                if detect_multiple_faces:
                    for i in range(nrof_faces):
                        det_arr.append(np.squeeze(det[i]))
                else:
                    bounding_box_size = (det[:,2]-det[:,0])*(det[:,3]-det[:,1])
                    img_center = img_size / 2
                    offsets = np.vstack([ (det[:,0]+det[:,2])/2-img_center[1], (det[:,1]+det[:,3])/2-img_center[0] ])
                    offset_dist_squared = np.sum(np.power(offsets,2.0),0)
                    index = np.argmax(bounding_box_size-offset_dist_squared*2.0) # some extra weight on the centering
                    det_arr.append(det[index,:])
            else:
                det_arr.append(np.squeeze(det))
            if len(det_arr)>0:
                faces = []
                bboxes = []
                for i, det in enumerate(det_arr):
                    det = np.squeeze(det)
                    bb = np.zeros(4, dtype=np.int32)
                    bb[0] = np.maximum(det[0]-margin/2, 0)
                    bb[1] = np.maximum(det[1]-margin/2, 0)
                    bb[2] = np.minimum(det[2]+margin/2, img_size[1])
                    bb[3] = np.minimum(det[3]+margin/2, img_size[0])
                    cropped = img[bb[1]:bb[3],bb[0]:bb[2],:]
                    scaled = misc.imresize(cropped, (image_size, image_size), interp='bilinear')
                    faces.append(scaled)
                    bboxes.append(bb)
                    
                return True,faces,bboxes


def identify_person(image_vector, feature_array, k=9):
        top_k_ind = np.argsort([np.linalg.norm(image_vector - pred_row) \
                        for ith_row, pred_row in enumerate(feature_array.values())])[:k]

        diff = np.linalg.norm(image_vector - list(feature_array.values())[top_k_ind[0]])
        #result = list(feature_array.keys())[top_k_ind[0]]
        result = []
        for i in range(k):
            result.extend([list(feature_array.keys())[top_k_ind[i]]])
            #print (np.linalg.norm(image_vector - list(feature_array.values())[top_k_ind[i]]), result[i])

        if (diff < 0.810):
            return result[0], diff
        else:
            return 'unknown/unknown_000.png', diff


def recognize_face (sess, pnet, rnet, onet,feature_array):
        # Get input and output tensors
        images_placeholder = sess.graph.get_tensor_by_name("input:0")
        images_placeholder = tf.image.resize_images(images_placeholder,(160,160))
        embeddings = sess.graph.get_tensor_by_name("embeddings:0")
        phase_train_placeholder = sess.graph.get_tensor_by_name("phase_train:0")

        embedding_size = embeddings.get_shape()[1]

        #img  = imageio.imread(imagePath)
        #gray = img

        #if (gray.size > 0):
        for img_file in img_files:
                gray      = imageio.imread(img_path + "/" + img_file)
                
                response, faces, bboxs = align_face(gray, pnet, rnet, onet)

                if (response == True):
                    for i, image in enumerate(faces):
                        bb = bboxs[i]
                        images = load_img(image, False, False, image_size)

                        feed_dict = { images_placeholder:images, phase_train_placeholder:False }
                        #feature_vector = sess.run(embeddings, feed_dict=feed_dict)
                        feature_vector = sess.run(embeddings, feed_dict)

                        result, diff = identify_person(feature_vector, feature_array, 5)
                        #print(result.split("/")[2])

                        W = int(bb[2]-bb[0])//2
                        H = int(bb[3]-bb[1])//2

                        if(result.split("/")[0] == 'unknown'):
                            cv2.rectangle(gray,(bb[0],bb[1]),(bb[2],bb[3]),(255,0,0),2)
                            cv2.rectangle(gray,(bb[0]-1,bb[1]),(bb[2]+1,bb[1]+26),(255,0,0),cv2.FILLED)
                            #cv2.rectangle(gray,(bb[0]-1,bb[3]),(bb[2]+1,bb[3]+18),(255,0,0),cv2.FILLED)
                            cv2.putText(gray, result.split("/")[0] + " " + str('%.2f' % diff),(bb[0],bb[1]+20), cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,191),1,cv2.LINE_AA)
                            #cv2.putText(gray, str(diff),(bb[0],bb[3]+10), cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1,cv2.LINE_AA)
                            #print("\x1b[1;31;40mUnknown " + "\x1b[0m" + result.split("/")[1])
                        else:
                            cv2.rectangle(gray,(bb[0],bb[1]),(bb[2],bb[3]),(0,255,0),2)
                            cv2.rectangle(gray,(bb[0]-1,bb[1]),(bb[2]+1,bb[1]+26),(0,255,0),cv2.FILLED)
                            #cv2.rectangle(gray,(bb[0]-1,bb[3]),(bb[2]+1,bb[3]+18),(0,255,0),cv2.FILLED)
                            cv2.putText(gray, result.split("/")[0] + " " + str('%.2f' % diff),(bb[0],bb[1]+20), cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,0),1,cv2.LINE_AA)
                            #cv2.putText(gray, str(diff),(bb[0],bb[3]+10), cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
                            #print("\x1b[1;32;40mMatched " + "\x1b[0m" + result.split("/")[1])
                            
                        img_face = gray[bb[1]:bb[3], bb[0]:bb[2]]        
                        plt.rcParams['figure.figsize'] = [5, 5]
                        plt.figure()
                        plt.axis('off')
                        plt.imshow(img_face)

                 
                    plt.rcParams['figure.figsize'] = [18, 18]
                    plt.figure()
                    plt.axis('off')
                    plt.imshow(gray)

# Recognize face
recognize_face (sess_tf, pnet, rnet, onet, feature_array)

PermissionError: [Errno 13] Permission denied: 'datasets/uit'